В данном ноутбуке мы парсим данные с сайта spymetrics. Эти данные представляют из собой: 
1. Посещаемость сайтов 
2. Показатель отказов (то, сколько людей заходят на одну страницу и выходят) 
3. Средняя продолжительность визита сайта 

Изначально мы выбрали примено 25 каналов, которые по нашему мнению являются наиболее привлекательными рекламными площадками. И хотим по этим 25 сайтам получить необходимую статистику для оценки эффективности каналов. Для парсинга сайтов мы используем библиотеку BeautifulSoup и регулярные выражения. 

In [9]:
import re
import ast
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format

import plotly.express as px
import datetime
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from datetime import datetime
import plotly.express as px

In [10]:
def parse_reject_time(website):
    """
    Возвращает показатель отказов, среднюю продолжительность визита на сайте
    ----------
    website: str
        Название сайта, который мы хотим проанализировать на эффективность.
    Returns
    -------
    reject_final: float
    time_final_new: float
        Показатель отказов
        Средняя продолжительность отказов
    """

  root_url = "https://spymetrics.ru/ru/website/" 
  full_url = root_url + website

  # создаем объект HTML сессии 
  session = HTMLSession()

  # Используем этот оюъект для соединения с нужной WEB-страницей
  html= session.get(full_url).text

  soup = BeautifulSoup(html, "html.parser")
  
  # Находим все теги, которые нам необходимы, чтобы сократить 
  list_js_scripts = soup.find_all("tbody")


  js_script_str = None
  for js_script in list_js_scripts:
      if "jQuery('#webcompareform')" in str(js_script):
          js_script_str = js_script.contents[0]

   # используем регулярные выражения для приближения к показателю отказов
  index_1= [match.start() for match in re.finditer("Показатель отказов", str(list_js_scripts[0]))][0]
  reject_1 = str(list_js_scripts[0])[index_1:]

  
  index_2 = [match.start() for match in re.finditer("text-right", reject_1)]
  index_end = [match.end() for match in re.finditer("</td>",reject_1)][1]
  a_new = reject_1[index_2[0]:index_end][12:]
  a_end = a_new[:-6]

  # получаем итоговый показатель отказов
  reject_final = float(a_end)
  index_1_time= [match.start() for match in re.finditer("Время на сайте", str(list_js_scripts[0]))][0]
  time_1 = str(list_js_scripts[0])[index_1_time:]
  index_2_time = [match.start() for match in re.finditer("text-right", time_1)]
  index_end_time = [match.end() for match in re.finditer("</td>",time_1)][1]
  a_new_time = time_1[index_2_time[0]:index_end_time][12:]

  # получаем время типа string, необходимо преобразоват в количество секунд
  time_final = a_new_time[:-5]
  import datetime 
  time1= datetime.time(0,0,0)
  from datetime import datetime, date
  time_object = datetime.strptime(time_final, '%H:%M:%S').time()
  time_final_new = datetime.combine(date.today(), time_object) - datetime.combine(date.today(), time1)
  return reject_final, time_final_new

In [12]:
def visits_data(website):
   """
    Рассчитывает количество визитов на сайте 
    ----------
    website: str
        Название сайта, который мы хотим проанализировать на эффективность.
    Returns
    -------
    d1['chart']['title']: string
    d1['series'][1]['data']: list
        Возвращает строку "Визитов".
        Возвращает список количества визитов ежемесячно.
    """

    root_url = "https://spymetrics.ru/ru/website/" 
    full_url = root_url + website

    # создаем HTML сессию
    session = HTMLSession()

    # Используем объект сессии для соединения с вебсайтом
    html= session.get(full_url).text
    soup = BeautifulSoup(html, "html.parser")
    list_js_scripts = soup.find_all("script", type="text/javascript")
    for js_script in list_js_scripts:
      if "jQuery('#webcompareform')" in str(js_script):
          js_script_str_ = js_script.contents[0]
    
    js_script_str = re.sub("Highcharts.Map", "Highcharts.Chart", js_script_str_)
    
    # Находим индексы Highcharts.Chart
    lst_1 = [match.start() for match in re.finditer("new Highcharts.Chart", js_script_str)][1:]
    lst_2 = [match.end() for match in re.finditer("new Highcharts.Chart", js_script_str)]
    
    # Пары индексов, находящихся между lst_1 и lst_2
    lst_tuples = list(zip(lst_2, lst_1))
    lst_lists = [list(elem) for elem in lst_tuples]
    
    # Избавляемся от мусора
    for t in lst_lists:
        t[0] = t[0] + 1
        t[1] = t[1] - 30
        
    # Извлекаем содержимое между Highcharts/Chart
    d1_str = js_script_str[lst_lists[0][0]:lst_lists[0][1]]
    d1_str = re.sub('false', "False", d1_str)
    d1_str = re.sub('null', '""', d1_str)
    re.sub("\\\\",'"', d1_str) ### careful, assignment may be needed!!!
    
    # Превращаем в словарь
    d1 = ast.literal_eval(d1_str)
    
    return d1['chart']['title'], d1['series'][1]['data']

В последующей функции мы рассчитываем эффективность. Эффективность рассчитывается по формуле: 
$$ Эффективность = \frac{\mbox{Средняя  продолжительность} \cdot \mbox{Количество визитов за последний месяц}}{\mbox{Показатель отказов}}$$

In [19]:
def plt_efficiency(file):
  """
    Строит графики по эффективности каналов в зависимости от кластера
    ----------
    file: str
        Название файла с результатами кластеризации
    Returns
    -------
    Два графика, потсроенных на одинковых данных, но в разном формате 
    """
  df_top5 = pd.read_csv(file)

  # создаем список с возможными сайтами
  websites = ['vk.ru', 'telegram.me', 'ok.ru', 'rbc.ru', 'vc.ru',  'forbes.ru', 'habr.com', 'knife.media', 'novayagazeta.ru', 'youtube.ru', 
  'ozon.ru', 'rutube.ru', 'avito.ru', 'cian.ru', 'youla.ru', 'wildberries.ru', 'wasd.tv', 'goodgame.ru', 'likee.video', 'otzovik.com', 
  'ivi.ru', 'kinopoisk.ru', '2gis.ru', 'fl.ru', 'netology.ru']
  efficiency = []

  # итерируемся по вебсайтам и рассчитывем среднюю продолжительность на сайте и визиты
  for i in websites:
    print(i)
    a = parse_reject_time(i)[1].total_seconds()
    b = parse_reject_time(i)[0]
    t = visits_data(i)[1][-1]
    efficiency.append(a * t/b)

  # создаем датафрейсм с сайтом и соответсвующей эффективностью 
  d = {'site': websites, 'efficiency': efficiency}
  df_media = pd.DataFrame(data=d)
  df_top_merged = df_top5.merge(df_media,on='site',how='left')
  df_top_merged['relative_efficiency'] = df_top_merged['efficiency']/df_top_merged['total_diff']
  print(df_top_merged)
  print(df_top_merged['efficiency'])
  print(df_top_merged['total_diff'])
  print(df_top_merged['cluster'])
  print(df_top_merged['site'])
  plt_topscat = px.scatter(df_top_merged,'efficiency','total_diff','cluster','site', title='Эффективность каналов на основе анализа ЦА банка и канала', labels={'x': 'Эффективность', 'y': 'Степень различия ЦА банка и канала'})
  plt_topbar = px.bar(df_top_merged,'site',['total_diff','efficiency','relative_efficiency'],facet_col='cluster',opacity=0.7, title='Анализ эффективности каналов', labels={'x': 'Сайт', 'y': 'Метрика'})
  return plt_topscat, plt_topbar